# Unit 3 服务 Part 2解决方案

<img src="../img/robotignite_logo_text.png"/>

## 索引: 
* <a href="#SolutionExercise3-2"> Exercise 3.2解决方案</a>
* <a href="#SolutionExercise3-3"> Exercise 3.3解决方案</a>

<p id="SolutionExercise3-2"></p>

## Exercise 3.2解决方案

* Exercise 3.2 的目的是创建一个服务，当被调用时，BB8机器人绕正方形轨迹运动。

Nothing to comment.

* 你可以创建并使用一个新的功能包，或者使用在Exercise 3.1中创建的那个名为**unit_3_services**的功能包。

本例中，我们将使用**unit_3_services**。

* 创建一个Python文件，内含一个允许BB8机器人在正方形内 <a href="#fig-3.1">{Fig-3.1}</a>运动的类。这个类可以命名为，例如**MoveBB8**。包含该类的python代码命名为 <a href="#move_bb8_py">move_bb8.py</a>。<br>
要使BB8机器人移动，你需要在 **/cmd_vel** 主题中发布消息，如在Unit1 主题中所做的那样。
需要注意的是，尽管这是仿真，但BB8机器人仍有重量，因此由于惯性它不能立即停下来。<br>
另外，当转弯时，摩擦和惯性将起作用。因此需要注意，仅通过/cmd_vel使机器人移动时，你无法检验它是否在按照你期望的方向转弯（这称为一个开环系统）。
当然，当你找到一种获取位置反馈信息的方法时，这则另当别论。这对先进的天文机械学家来说是一个挑战（如果你想尝试的话，可考虑使用/odom主题）。<br>
对于本例，只要机器人能绕正方形内运动即可，无须追求完美。


首先，我们在功能包**unit_3_services**中创建python文件 **move_bb8.py** 。你可以把它放在**scripts**文件夹或者 **src**文件夹中。这取决于你是否打算将这个类作为一个Python模块以便允许该功能包外的其他功能包访问它。<br>
如果你想使它在功能包**unit_3_services**外被使用，建议把它放在src文件夹中。由于这些超出了本单元的讨论范围，我们仍把它放在**scripts**中。在 **sphero 项目**中你将会学到更加复杂的方法。

下面是拥有**MoveBB8**类的python文件代码，该文件以开环的方式使BB8绕正方形轨迹移动。

<p style="background:#3B8F10;color:white;" id="move_bb8_py">**Python Program: move_bb8.py** </p><br>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
import time

class MoveBB8():
    
    def __init__(self):
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
    
    
    def publish_once_in_cmd_vel(self, cmd):
        """
        This is because publishing in topics sometimes fails teh first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
    
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            self.stop_bb8()
            self.ctrl_c = True

    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)


    def move_x_time(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = angular_speed
        
        rospy.loginfo("Moving Forwards")
        self.publish_once_in_cmd_vel(cmd)
        time.sleep(moving_time)
        self.stop_bb8()
        rospy.loginfo("######## Finished Moving Forwards")
    
    def move_square(self):
        
        i = 0
        while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0, linear_speed=0.2, angular_speed=0.0)
            # Stop
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.0)
            # Turn 
            self.move_x_time(moving_time=3.5, linear_speed=0.0, angular_speed=0.2)
            # Stop
            self.move_x_time(moving_time=0.1, linear_speed=0.0, angular_speed=0.0)
            
            i += 1
        rospy.loginfo("######## Finished Moving in a Square")
        

            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        movebb8_object.move_square()
    except rospy.ROSInterruptException:
        pass


<p style="background:#3B8F10;color:white;" id="move_bb8_py">END **Python Program: move_bb8.py** </p><br>

* 添加一个消息服务器接收一个 <b>Empty</b> 服务消息，激活正方形运动。该服务可以命名为 **/move_bb8_in_square**<br>
激活通过调用类**MoveBB8**来完成。<br>
要做到这一点，你需要创建一个类似的python文件 <a href="#prg-3-7">simple_service_server.py</a>。你可以把它称为**bb8_move_in_square_service_server.py**。

因此第一步是创建一个名为 <a href="#bb8_move_in_square_service_server_py">bb8_move_in_square_service_server.py</a>的python文件, 它可以看做 **example 3.7**的一个修改版。

<p style="background:#3B8F10;color:white;" id="bb8_move_in_square_service_server_py">**Python Program: bb8_move_in_square_service_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Empty, EmptyResponse # you import the service message python classes generated from Empty.srv.
from move_bb8 import MoveBB8

def my_callback(request):
    rospy.loginfo("The Service move_bb8_in_square has been called")
    movebb8_object = MoveBB8()
    movebb8_object.move_square()
    rospy.loginfo("Finished service move_bb8_in_square that was called called")
    return EmptyResponse() # the service Response class, in this case EmptyResponse
    #return MyServiceResponse(len(request.words.split())) 

rospy.init_node('service_move_bb8_in_square_server') 
my_service = rospy.Service('/move_bb8_in_square', Empty , my_callback) # create the Service called move_bb8_in_square with the defined callback
rospy.loginfo("Service /move_bb8_in_square Ready")
rospy.spin() # mantain the service open.

<p style="background:#3B8F10;color:white;">END **Python Program: bb8_move_in_square_service_server.py** </p>

完成后，在WebConsole中**调用** 服务 **/move_bb8_in_square** 进行测试。

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
rosservice call /move_bb8_in_square [TAB]+[TAB]

BB8机器人运动如下：

<img src="../img/basic_unit3_exercise3-2.gif"/>

* 创建一个名为 **start_bb8_move_in_square_service_server.launch**的launch文件。在该文件中，开启一个启动**bb8_move_in_square_service_server.py** 的节点。

下面是启动 **bb8_move_in_square_service_server.py** 的launch文件<a href="#start_bb8_move_in_square_service_server_launch">start_bb8_move_in_square_service_server.launch</a> 。

<p style="background:#3B8F10;color:white;" id="start_bb8_move_in_square_service_server_launch">**Launch Program: start_bb8_move_in_square_service_server.launch** </p>

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="unit_3_services" type="bb8_move_in_square_service_server.py" name="service_move_bb8_in_square_server"  output="screen">
    </node>
</launch>

<p style="background:#3B8F10;color:white;">END **Launch Program: start_bb8_move_in_square_service_server.launch** </p>

如前面介绍的那样，启动**start_bb8_move_in_square_service_server.launch**并调用该服务进行测试。步骤完全相同，唯一不同的是这里我们是通过一个launch文件而非python文件来启动服务服务器的。

* 创建一个新的python节点，命名为 **bb8_move_in_square_service_client.py**, 该节点调用服务 **/move_bb8_in_square**。回顾一下  **Unit3 服务 Part1** 中它是如何实现的。<br>
然后生成一个新的名为 **call_bb8_move_in_square_service_server.launch** 的launch文件, 它通过一个节点运行**bb8_move_in_square_service_client.py** 。

<a href="#bb8_move_in_square_service_client_py">bb8_move_in_square_service_client.py</a> 看起来应该是这样的：

<p style="background:#3B8F10;color:white;" id="bb8_move_in_square_service_client_py">**Python Program: bb8_move_in_square_service_client.py** </p>

In [ ]:
#! /usr/bin/env python
import rospkg
import rospy
from std_srvs.srv import Empty, EmptyRequest # you import the service message python classes generated from Empty.srv.

rospy.init_node('service_move_bb8_in_square_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/move_bb8_in_square') # Wait for the service client /move_bb8_in_square to be running
move_bb8_in_square_service_client = rospy.ServiceProxy('/move_bb8_in_square', Empty) # Create the connection to the service
move_bb8_in_square_request_object = EmptyRequest() # Create an object of type EmptyRequest

result = move_bb8_in_square_service_client(move_bb8_in_square_request_object) # Send through the connection the path to the trajectory file to be executed
print result # Print the result given by the service called

<p style="background:#3B8F10;color:white;">END **Python Program: bb8_move_in_square_service_client.py** </p>

<a href="#call_bb8_move_in_square_service_server_launch">call_bb8_move_in_square_service_server.launch</a> 看起来应该是这样的：

<p style="background:#3B8F10;color:white;" id="call_bb8_move_in_square_service_server_launch">**Launch Program: call_bb8_move_in_square_service_server.launch** </p>

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="unit_3_services" type="bb8_move_in_square_service_client.py" name="service_move_bb8_in_square_client"  output="screen">
    </node>
</launch>

<p style="background:#3B8F10;color:white;" id="start_bb8_move_in_square_service_server_launch">**Launch Program: call_bb8_move_in_square_service_server.launch** </p>

* 当启动 **call_bb8_move_in_square_service_server.launch**时, BB8机器人将在一个正方形内移动。

工作原理与你前面进行的其他调用完全相同。在本例中，启动 **start_bb8_move_in_square_service_server.launch** ，然后在另一个终端启动 **call_bb8_move_in_square_service_server.launch**。

<p id="SolutionExercise3-3"></p>

## Exercise 3.3解决方案

* 升级python文件 **move_bb8.py** ，使BB8绕一个大小变化的正方形轨迹移动。

修改<a href="#move_bb8_py_mod">move_bb8.py</a> 文件如下：

<p style="background:#3B8F10;color:white;" id="move_bb8_py_mod">**Python File: move_bb8.py** </p>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
import time
import math

class MoveBB8():
    
    def __init__(self):
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
    
    
    def publish_once_in_cmd_vel(self, cmd):
        """
        This is because publishing in topics sometimes fails teh first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
    
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            self.stop_bb8()
            self.ctrl_c = True

    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)


    def move_x_time(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = angular_speed
        
        rospy.loginfo("Moving Forwards")
        self.publish_once_in_cmd_vel(cmd)
        time.sleep(moving_time)
        self.stop_bb8()
        rospy.loginfo("######## Finished Moving Forwards")
    
    def move_square(self, side=0.2):
        
        i = 0
        # More Speed, more time to stop
        time_magnitude = side / 0.2
        
        while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0*time_magnitude, linear_speed=0.2, angular_speed=0.0)
            # Stop
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.0)
            # Turn, the turning is not afected by the length of the side we want
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.2)
            # Stop
            self.move_x_time(moving_time=0.1, linear_speed=0.0, angular_speed=0.0)
            
            i += 1
        rospy.loginfo("######## Finished Moving in a Square")
        

            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        movebb8_object.move_square(side=0.6)
    except rospy.ROSInterruptException:
        pass


<p style="background:#3B8F10;color:white;">END **Python File: move_bb8.py** </p>

如你所见，代码的变动很少：

In [ ]:
while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0*time_magnitude, linear_speed=0.2, angular_speed=0.0)
            # Stop

我们只是改变了BB8机器人向前移动的时间，以此来控制该机器人正方形运动轨迹的大小。当然，在没有校正和闭环控制的情况下，很难使BB8走出一个完美的方形。但这没有关系，我们的目的是改变正方形的大小，不要求很精确。

* 创建一个新的名为 **bb8_move_custom_service_server.py** 的python文件, 修改你在Exercise 3.2中创建的接收空服务消息并激活正方形运动的服务服务器。这个新的服务可以命名为 **/move_bb8_in_square_custom**，它使用类型为**BB8CustomServiceMessage** 的服务消息。该服务消息定义如下： 

首先创建 <a href="#BB8CustomServiceMessage_srv">BB8CustomServiceMessage.srv</a>, 在功能包 **unit_3_services** 中创建一个**srv** 文件夹。

<p style="background:#3B8F10;color:white;" id="BB8CustomServiceMessage_srv">**Service Message: BB8CustomServiceMessage.srv** </p>

In [ ]:
float64 side       # The distance of each side of the square
int32 repetitions    # The number of times BB-8 has to execute the square movement when the service is called
---
bool success         # Did it achieve it?

<p style="background:#3B8F10;color:white;" id="BB8CustomServiceMessage_srv">END **Service Message: BB8CustomServiceMessage.srv** </p>

修改 <a href="#CMakelists_txt">CMakelists.txt</a> 和 <a href="#package_xml">package.xml</a>

<p style="background:#3B8F10;color:white;" id="CMakelists_txt">**CMakelists.txt** </p>

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(unit_3_services)


## Here go all the packages needed to COMPILE the messages of topic, services and actions.
## Its only geting its paths, and not really importing them to be used in the compilation.
## Its only for further functions in CMakeLists.txt to be able to find those packages.
## In package.xml you have to state them as build
find_package(catkin REQUIRED COMPONENTS
  std_msgs
  message_generation
)

## Generate services in the 'srv' folder
## In this function will be all the action messages of this package ( in the action folder ) to be compiled.
## You can state that it gets all the actions inside the action directory: DIRECTORY action
## Or just the action messages stated explicitly: FILES my_custom_action.action
## In your case you only need to do one of two things, as you wish.
add_service_files(
  FILES
  BB8CustomServiceMessage.srv
)

## Here is where the packages needed for the action messages compilation are imported.
generate_messages(
  DEPENDENCIES
  std_msgs
)

## State here all the packages that will be needed by someone that executes something from your package.
## All the packages stated here must be in the package.xml as run_depend
catkin_package(
  CATKIN_DEPENDS rospy
)


include_directories(
  ${catkin_INCLUDE_DIRS}
)

<p style="background:#3B8F10;color:white;" id="package_xml">**package.xml** </p>

In [ ]:
<?xml version="1.0"?>
<package>
  <name>unit_3_services</name>
  <version>0.0.0</version>
  <description>The unit_3_services package</description>

  <maintainer email="user@todo.todo">user</maintainer>

  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>rospy</build_depend>
  <build_depend>message_generation</build_depend>
  <run_depend>rospy</run_depend>
  <run_depend>message_runtime</run_depend>


  <!-- The export tag contains other, unspecified, tags -->
  <export>
    <!-- Other tools can request additional information be placed here -->
  </export>
</package>

完成后，进行编译和source，以便ROS能找到新的消息。

In [ ]:
roscd;cd ..
catkin_make
source devel/setup.bash

检查是否能发现该消息：

In [ ]:
rossrv list | grep BB8CustomServiceMessage

你将得到：

In [ ]:
unit_3_services/BB8CustomServiceMessage

* 使用传递给新 **/move_bb8_in_square_custom** 的数据来改变BB8运动的方式。<br>
根据 **side** 值，该服务将使BB8机器人绕一个大小为**side**的正方形轨迹运动。<br>
另外，BB8机器人必须重复该方形运动，重复次数由该消息中**repetitions** 变量给出。<br> 最后，如果一切正常，它必须在**success**变量中返回真值。

现在，使用新的服务消息**BB8CustomServiceMessage**创建 <a href="#bb8_move_custom_service_server.py">bb8_move_custom_service_server.py</a>。

<p style="background:#3B8F10;color:white;" id="bb8_move_custom_service_server.py">**Python File: bb8_move_custom_service_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from unit_3_services.srv import BB8CustomServiceMessage, BB8CustomServiceMessageResponse
from move_bb8 import MoveBB8

"""
# BB8CustomServiceMessage
float64 side       # The distance of each side of the square
int32 repetitions    # The number of times BB-8 has to execute the square movement when the service is called
---
bool success         # Did it achieve it?
"""

def my_callback(request):
    rospy.loginfo("The Service move_bb8_in_square_custom has been called")
    
    movebb8_object = MoveBB8()
    
    # We need to add because if we ask 0 repetitions means to execute once, not zero times.
    repetitions_of_square = request.repetitions + 1
    new_side = request.side
    for i in range(repetitions_of_square):
        rospy.loginfo("Start Movement with side = "+str(new_side)+"Repetition = "+str(i))
        movebb8_object.move_square(side=new_side)
        
    rospy.loginfo("Finished service move_bb8_in_square that was called called")
    response = BB8CustomServiceMessageResponse()
    response.success = True
    return response

rospy.init_node('service_move_bb8_in_square_custom_server') 
my_service = rospy.Service('/move_bb8_in_square_custom', BB8CustomServiceMessage , my_callback) # create the Service called move_bb8_in_square with the defined callback
rospy.loginfo("Service /move_bb8_in_square_custom Ready")
rospy.spin() # mantain the service open.

<p style="background:#3B8F10;color:white;">END **Python File: bb8_move_custom_service_server.py** </p>

* 创建一个新的名为**start_bb8_move_custom_service_server.launch**的launch文件，启动python文件**bb8_move_custom_service_server.py**中的服务器。
* 测试一下，当调用这个新的服务 **/move_bb8_in_square_custom**时, BB8机器人进行相应地运动。 

**start_bb8_move_custom_service_server.launch** 与Exercise 3.2中的那个相同，只是将启动的python文件改成了 **bb8_move_custom_service_server.py**。

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="unit_3_services" type="bb8_move_custom_service_server.py" name="service_move_bb8_in_square_custom_server"  output="screen">
    </node>
</launch>

* 创建一个新的服务客户端，调用服务 **/move_bb8_in_square_custom** ，使BB8机器人绕一个小的正方形 **两次**，绕一个大的正方形 **一次**。该服务器客户端可以命名为 **bb8_move_custom_service_client.py** ，启动它的launch文件命名为**call_bb8_move_in_square_custom_service_server.launch**。

创建执行两次小正方形运动和一次大正方形运动的 python文件<a href="#bb8_move_custom_service_client_py">bb8_move_custom_service_client.py</a> 。

<p style="background:#3B8F10;color:white;" id="bb8_move_custom_service_client_py">**Python File: bb8_move_custom_service_client.py** </p>

In [ ]:
#! /usr/bin/env python
import rospkg
import rospy
from unit_3_services.srv import BB8CustomServiceMessage, BB8CustomServiceMessageRequest


rospy.init_node('service_move_bb8_in_square_custom_client') # Initialise a ROS node with the name service_client
rospy.wait_for_service('/move_bb8_in_square_custom') # Wait for the service client /move_bb8_in_square to be running
move_bb8_in_square_service_client = rospy.ServiceProxy('/move_bb8_in_square_custom', BB8CustomServiceMessage) # Create the connection to the service
move_bb8_in_square_request_object = BB8CustomServiceMessageRequest() # Create an object of type EmptyRequest


"""
# BB8CustomServiceMessage
float64 side       # The distance of each side of the square
int32 repetitions    # The number of times BB-8 has to execute the square movement when the service is called
---
bool success         # Did it achieve it?
"""
move_bb8_in_square_request_object.side = 0.1
move_bb8_in_square_request_object.repetitions = 2

rospy.loginfo("Start Two Small Squares...")
result = move_bb8_in_square_service_client(move_bb8_in_square_request_object) # Send through the connection the path to the trajectory file to be executed
rospy.loginfo(str(result)) # Print the result given by the service called

move_bb8_in_square_request_object.side = 0.6
move_bb8_in_square_request_object.repetitions = 1

rospy.loginfo("Start One Big Square...")
result = move_bb8_in_square_service_client(move_bb8_in_square_request_object) # Send through the connection the path to the trajectory file to be executed
rospy.loginfo(str(result))

rospy.loginfo("END of Service call...")

<p style="background:#3B8F10;color:white;">END **Python File: bb8_move_custom_service_client.py** </p>

现在，创建一个launch文件启动该python节点。该launch文件命名为 <a href="#call_bb8_move_in_square_custom_service_server_launch">call_bb8_move_in_square_custom_service_server.launch</a>。

<p style="background:#3B8F10;color:white;" id="call_bb8_move_in_square_custom_service_server_launch">**Launch File: call_bb8_move_in_square_custom_service_server.launch** </p>

In [ ]:
<launch>
    <!-- Start Service Server for move_bb8_in_square service -->
    <node pkg="unit_3_services" type="bb8_move_custom_service_client.py" name="service_move_bb8_in_square_custom_client"  output="screen">
    </node>
</launch>

<p style="background:#3B8F10;color:white;">END **Launch File: call_bb8_move_in_square_custom_service_server.launch** </p>

最后，测试整个管线。在一个web终端启动服务器，在另一个终端启动客户端。BB8机器人将按照期望的方式运动。

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch unit_3_services start_bb8_move_custom_service_server.launch

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>
<br>

In [ ]:
roslaunch unit_3_services call_bb8_move_in_square_custom_service_server.launch

你将看到类似下面的结果，但是会慢一些，因为这里进行了快放处理：

<img src="../img/basic_unit3_exercise3-3_final.gif"/>

## 里程计解决方案:

下面是一个如何基于里程计读数访问和移动机器人的例子。注意，里程计并不完美，因此存在一些不确定性，特别是在方向方面。但它的位置很精确。这也是为什么机器人需要其他定位系统的原因。如果你感兴趣，请学习**ROS navigation** 课程，或者 **Jackal robot** 或 **Summit XL** 的机器人课程。

<p style="background:#3B8F10;color:white;" id="UPGRADE_move_bb8_py">UPGRADE **Python File: move_bb8.py** </p>

In [ ]:
#!/usr/bin/env python
import rospy
from geometry_msgs.msg import Twist
import time
import math
from nav_msgs.msg import Odometry
import tf

"""
rosmsg show nav_msgs/Odometry
std_msgs/Header header
  uint32 seq
  time stamp
  string frame_id
string child_frame_id
geometry_msgs/PoseWithCovariance pose
  geometry_msgs/Pose pose
    geometry_msgs/Point position
      float64 x
      float64 y
      float64 z
    geometry_msgs/Quaternion orientation
      float64 x
      float64 y
      float64 z
      float64 w
  float64[36] covariance
geometry_msgs/TwistWithCovariance twist
  geometry_msgs/Twist twist
    geometry_msgs/Vector3 linear
      float64 x
      float64 y
      float64 z
    geometry_msgs/Vector3 angular
      float64 x
      float64 y
      float64 z
  float64[36] covariance
"""


class OdomTools(object):
    def __init__(self):
        self.init_odom()
        self.odom_subscriber = rospy.Subscriber("/odom", Odometry, self.odom_callback)

    def init_odom(self):
        self._odometry = None
        while self._odometry is None:
            try:                
                self._odometry = rospy.wait_for_message("/odom", Odometry, timeout=1)
            except:
                rospy.loginfo("/odom topic is not ready yet, retrying")
        
        rospy.loginfo("/odom topic READY")
        

    def odom_callback(self,msg):
        self._odometry = msg
    
    def get_odom_position(self):
        return self._odometry.pose.pose.position
        
    def get_odom_orientation(self):
        return self._odometry.pose.pose.orientation
    
    def calculate_distance_change(self,init_point, finish_point):
        
        delta_x = finish_point.x - init_point.x
        delta_y = finish_point.y - init_point.y
        delta_z = finish_point.z - init_point.z
        
        delta_x2 = pow(delta_x,2)
        delta_y2 = pow(delta_y,2)
        delta_z2 = pow(delta_z,2)
        
        distance = math.sqrt(delta_x2+delta_y2+delta_z2)
        
        return distance
        
    def calculate_yaw_angle_change(self,init_orientation, finish_orientation):

        explicit_init_quat = [init_orientation.x, init_orientation.y, init_orientation.z, init_orientation.w]
        explicit_finish_quat = [finish_orientation.x, finish_orientation.y, finish_orientation.z, finish_orientation.w]

        init_euler = tf.transformations.euler_from_quaternion(explicit_init_quat)
        finish_euler = tf.transformations.euler_from_quaternion(explicit_finish_quat)

        delta_yaw=finish_euler[2]- init_euler[2]
        
        return delta_yaw

class MoveBB8():
    
    def __init__(self):
        self.odom_object = OdomTools()
        self.bb8_vel_publisher = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.ctrl_c = False
        rospy.on_shutdown(self.shutdownhook)
        self.rate = rospy.Rate(10) # 10hz
    
        
    
    def publish_once_in_cmd_vel(self, cmd):
        """
        This is because publishing in topics sometimes fails teh first time you publish.
        In continuos publishing systems there is no big deal but in systems that publish only
        once it IS very important.
        """
        while not self.ctrl_c:
            connections = self.bb8_vel_publisher.get_num_connections()
            if connections > 0:
                self.bb8_vel_publisher.publish(cmd)
                rospy.loginfo("Cmd Published")
                break
            else:
                self.rate.sleep()
    
    
    def shutdownhook(self):
            # works better than the rospy.is_shut_down()
            self.stop_bb8()
            self.ctrl_c = True

    def stop_bb8(self):
        rospy.loginfo("shutdown time! Stop the robot")
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)


    def move_x_time(self, moving_time, linear_speed=0.2, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = angular_speed
        
        rospy.loginfo("Moving Forwards")
        self.publish_once_in_cmd_vel(cmd)
        time.sleep(moving_time)
        self.stop_bb8()
        rospy.loginfo("######## Finished Moving Forwards")
    
    def move_square(self, side=0.2):
        
        i = 0
        # More Speed, more time to stop
        time_magnitude = side / 0.2
        
        while not self.ctrl_c and i < 4:
            # Move Forwards
            self.move_x_time(moving_time=2.0*time_magnitude, linear_speed=0.2, angular_speed=0.0)
            # Stop
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.0)
            # Turn, the turning is not afected by the length of the side we want
            self.move_x_time(moving_time=4.0, linear_speed=0.0, angular_speed=0.2)
            # Stop
            self.move_x_time(moving_time=0.1, linear_speed=0.0, angular_speed=0.0)
            
            i += 1
        rospy.loginfo("######## Finished Moving in a Square")
        
    def move_distance(self, distance_to_move,linear_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = linear_speed
        cmd.angular.z = 0.0
        
        
        init_position = self.odom_object.get_odom_position()
        final_position = self.odom_object.get_odom_position()
        distance_moved = self.odom_object.calculate_distance_change(init_position,final_position)
        
        while distance_moved < distance_to_move:
            rospy.loginfo("Moving")
            self.publish_once_in_cmd_vel(cmd)
            self.rate.sleep()
            final_position = self.odom_object.get_odom_position()
            distance_moved = self.odom_object.calculate_distance_change(init_position,final_position)
            rospy.loginfo("Distance Moved::>"+str(distance_moved))
        
        rospy.loginfo("Stopping...")
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)
        
    def move_angle(self, angle_to_turn, angular_speed=0.2):
        
        cmd = Twist()
        cmd.linear.x = 0.0
        cmd.angular.z = angular_speed
        
        init_orientation = self.odom_object.get_odom_orientation()
        finish_orientation = self.odom_object.get_odom_orientation()
        delta_yaw = self.odom_object.calculate_yaw_angle_change(init_orientation, finish_orientation)
        
        while delta_yaw < angle_to_turn:
            rospy.loginfo("Turning")
            self.publish_once_in_cmd_vel(cmd)
            self.rate.sleep()
            finish_orientation = self.odom_object.get_odom_orientation()
            delta_yaw = self.odom_object.calculate_yaw_angle_change(init_orientation, finish_orientation)
            rospy.loginfo("Angle Moved::>"+str(delta_yaw))
        
        rospy.loginfo("Stopping...")
        cmd.linear.x = 0.0
        cmd.angular.z = 0.0
        self.publish_once_in_cmd_vel(cmd)
    
    def calibrate_turning(self):
        
        init_orientation = self.odom_object.get_odom_orientation()
        while not self.ctrl_c:
            
            finish_orientation = self.odom_object.get_odom_orientation()
            delta_yaw = self.odom_object.calculate_yaw_angle_change(init_orientation, finish_orientation)
            rospy.loginfo("Angle Moved::>"+str(delta_yaw))
            self.rate.sleep()
        
        
            
if __name__ == '__main__':
    rospy.init_node('move_bb8_test', anonymous=True)
    movebb8_object = MoveBB8()
    try:
        #movebb8_object.move_square(side=0.6)
        #movebb8_object.move_cicle(radius=20.0)
        movebb8_object.move_distance(distance_to_move=1.0,linear_speed=0.2)
        
    except rospy.ROSInterruptException:
        pass


<p style="background:#3B8F10;color:white;" >END UPGRADE **Python File: move_bb8.py** </p>

如你所见，BB8机器人运动和定位的精度不是很高。当然，使用PID控制来到达精确的位置和方位将会好一些，但这不是本课程所要讨论的。如果你感兴趣，请学习**Perception** 课程，该课程中将讨论使用**pid** ROS功能包来实现该目的。